In [10]:
# import libraries
import requests
import time
from datetime import datetime, date, timedelta
import json
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)  #allows display of all columns in dfs


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
colors = ["#118AB2", "#EF476F", "#FFD166", "#06D6A0", "#EE754D", "#002E99"]
#plt.style.use('seaborn-whitegrid')

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [11]:
# mysql login info
sys.path.insert(0, '../../Key')
from mysql_secret import dbuser, dbpass, dbhost, dbname
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')

In [12]:
def flatten_cols(df):
    df.columns = [
        '_'.join(tuple(map(str, t))).strip() 
        for t in df.columns.values
        ]
    return df

In [13]:
# Establish Twitter connection
import tweepy

# get tweepy credentials
sys.path.insert(0, '../../Key')
from tweepy_keys import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_KEY, ACCESS_SECRET

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth)

# Upcoming Week Schedule

In [14]:
# get range for today until one week from today
today = date.today()
week_from_today = today + timedelta(days=7)

#convert to string for query purposes from SQL
today = today.strftime("%Y-%m-%d")
week_from_today = week_from_today.strftime("%Y-%m-%d")

print(f"{today} to {week_from_today}")

2021-02-01 to 2021-02-08


In [15]:
connection = engine.connect()
this_weeks_ipos_df = pd.read_sql(f"""
    SELECT *
    FROM stocks
    WHERE priced_date BETWEEN '{today}' and '{week_from_today}'
    """, connection)

In [16]:
# add trimmed date that is only month and day
this_weeks_ipos_df['priced_date'] =  pd.to_datetime(this_weeks_ipos_df['priced_date'])

# add column for month day
this_weeks_ipos_df["price_month_day"] = this_weeks_ipos_df['priced_date'].dt.strftime('%b %d')
this_weeks_ipos_df = this_weeks_ipos_df.sort_values("priced_date")
this_weeks_ipos_df

,symbol,company,exchange,proposed_share_price,shares_offered,priced_date,dollar_val_shares,deal_status,proposed_cap_classifiction,price_month_day
3,ONTF,ON24 INC,NYSE,45.00-50.00,8600977,2021-02-03,494556150.0,expected,Small-cap,Feb 03
5,TIXT,TELUS International (Cda) Inc.,NYSE,23.00-25.00,33333333,2021-02-03,958333325.0,expected,Small-cap,Feb 03
0,ATC,Atotech Ltd,NYSE,19.00-22.00,34146000,2021-02-04,863893800.0,expected,Small-cap,Feb 04
1,LABP,"Landos Biopharma, Inc.",NASDAQ Global,15.00-17.00,6250000,2021-02-04,122187500.0,expected,Micro-cap,Feb 04
2,LDI,"loanDepot, Inc.",NYSE,19.00-21.00,15000000,2021-02-04,362250000.0,expected,Small-cap,Feb 04
4,SANA,"Sana Biotechnology, Inc.",NASDAQ Global Select,20.00-23.00,15000000,2021-02-04,396750000.0,expected,Small-cap,Feb 04


In [17]:
# get tweet text 
tweet_text = "Upcoming scheduled IPOs:\n"
for row in this_weeks_ipos_df.itertuples():
    
    # remove .00 from proposed share prices to reduce unecesary characters
    psp = row.proposed_share_price.replace(".00", "")
    
    tweet_text += "\n"
    row_text = f"{row.price_month_day}: ${row.symbol} proposed price ${psp}" 
    tweet_text += row_text
print(tweet_text)
print(f"******\nCharacter Count: {len(tweet_text)}")


Upcoming scheduled IPOs:

Feb 03: $ONTF proposed price $45-50
Feb 03: $TIXT proposed price $23-25
Feb 04: $ATC proposed price $19-22
Feb 04: $LABP proposed price $15-17
Feb 04: $LDI proposed price $19-21
Feb 04: $SANA proposed price $20-23
******
Character Count: 239


In [18]:
# Send out the tweet
api.update_status(tweet_text)  

Status(_api=<tweepy.api.API object at 0x000001E2943D0E48>, _json={'created_at': 'Mon Feb 01 21:24:06 +0000 2021', 'id': 1356352649571020800, 'id_str': '1356352649571020800', 'text': 'Upcoming scheduled IPOs:\n\nFeb 03: $ONTF proposed price $45-50\nFeb 03: $TIXT proposed price $23-25\nFeb 04: $ATC prop… https://t.co/DtDn26SNlZ', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [{'text': 'ONTF', 'indices': [34, 39]}, {'text': 'TIXT', 'indices': [70, 75]}, {'text': 'ATC', 'indices': [106, 110]}], 'user_mentions': [], 'urls': [{'url': 'https://t.co/DtDn26SNlZ', 'expanded_url': 'https://twitter.com/i/web/status/1356352649571020800', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1285746488862269440, 'id_str': '1285746488862269440', 'name': 'IPO Tracker 3000', 'screen_na

# Stock Goes Live Today

In [ ]:
# get todays date
today = date(2021, 1, 21).strftime("%Y-%m-%d")  #date for testing, not actually today
# today = datetime.today().strftime("%Y-%m-%d")
print(today)

In [ ]:
# get stocks that go live today
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')
connection = engine.connect()
df = pd.read_sql(f"""
    SELECT s.*, 
        ci.city, 
        ci.state,
        ci.country,
        ci.website, 
        ci.industry,
        ci.sector, 
        ci.business_summary
    FROM stocks s
    LEFT JOIN company_info ci
        ON s.symbol = ci.symbol
    WHERE priced_date = '{today}'
        AND exchange <> 'NASDAQ Capital'""", connection)
connection.close()
df

In [ ]:
# industry portion of tweet
stock_industry = None
if df['industry'][0] is None:
    pass
elif df['industry'][0] == '':
    pass
else:
    stock_industry = df['industry'][0].lower()

    
print(stock_industry)

In [ ]:
# region portion of tweet
stock_region = None

# if no country information, pass
if df['country'][0] is None:
    pass
elif df['industry'][0] == '':
    pass
else:
    stock_industry = df['industry'][0].lower()

    
print(stock_region)

In [ ]:
tweet_text = f"{df['company'][0]} ${df['symbol'][0]} should trade today, a {stock_industry} company out of "

tweet_text

In [ ]:
# wrap text https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/autowrap.html

# Performance Analysis

In [ ]:
connection = engine.connect()
ipo_df = pd.read_sql("SELECT * FROM ipo_tracker.vw_stocks_performance", connection)
connection.close()
ipo_df.head()

## Sector and Industry

In [ ]:
# remove stocks with None or blank as sector / industry
si_data_df = ipo_df[(ipo_df["industry"].notnull()) & (ipo_df["industry"] != "")
            & (ipo_df["sector"].notnull()) & (ipo_df["sector"] != "")]
si_data_df.head(2)

### Sector by Year

In [ ]:
sector_by_year_df = si_data_df.pivot_table(index='sector', columns='first_trade_year', 
                                    aggfunc={"symbol": 'count'})
sector_by_year_df = sector_by_year_df.fillna(0).sort_values(by=sector_by_year_df.columns[-1], ascending=True)

# pass to function to flatten columns
sector_by_year_df = flatten_cols(sector_by_year_df)
sector_by_year_df


In [ ]:
ax = sector_by_year_df.plot(kind="barh", figsize=(8,7), color=colors, zorder=3, width=0.75)
ax.grid(which="major", axis='both', color='#e6e6e6', zorder=0)
ax.legend([2018, 2019, 2020])
ax.set_ylabel(f'Sector', fontsize=12)
ax.set_xlabel(f'IPOs Count', fontsize=12)
ax.set_title("IPO Count by Sector", fontsize=20)
plt.show()

## Top Performing n Period

Values for analysis

In [ ]:
# example stock to view
stock_symbol = 'ABCM' 

Data preparation

In [ ]:
# add price change column
sa_df = ipo_df.copy()
sa_df["first_close_to_rec_close"] = sa_df["most_recent_close"] / sa_df["first_day_close"] - 1
sa_df.head(5)

In [ ]:
# get sector for stock of interest
stock_sector = sa_df.loc[sa_df["symbol"] == stock_symbol]["sector"].iloc[0]
print(f"{stock_symbol} is in the {stock_sector} sector")

# get industry for stock of interest
stock_industry = sa_df.loc[sa_df["symbol"] == stock_symbol]["industry"].iloc[0]
print(f"{stock_symbol} is in the {stock_industry} industry")

# get market cap classification for stock of interest
stock_market_cap = sa_df.loc[sa_df["symbol"] == stock_symbol]["market_cap_classification"].iloc[0]
print(f"{stock_symbol} is a {stock_market_cap} market cap")

In [ ]:
# create new columns for vizualization
# sector
conditions = [(sa_df["symbol"] == stock_symbol), (sa_df["sector"] == stock_sector)]
choices = [stock_symbol, stock_sector]
sa_df["Stock Sector"] = np.select(conditions, choices, "Other")

# industry
conditions = [(sa_df["symbol"] == stock_symbol), (sa_df["industry"] == stock_industry)]
choices = [stock_symbol, stock_industry]
sa_df["Stock Industry"] = np.select(conditions, choices, "Other")

# market cap
conditions = [(sa_df["symbol"] == stock_symbol), (sa_df["industry"] == stock_market_cap)]
choices = [stock_symbol, stock_market_cap]
sa_df["Stock Market Cap"] = np.select(conditions, choices, "Other")

sa_df.head(2)

In [ ]:
# data for plotting sector in similar market cap
sa_mc_df = sa_df[sa_df["market_cap_classification"] == stock_market_cap]
sa_mc_df

In [ ]:
# Plot
colors = ["#118AB2", "#EF476F", "#d9d9d9"]
sns.lmplot('days_trading', 'first_close_to_rec_close', data=sa_mc_df, hue='Stock Sector', 
           palette=colors[0:3], scatter_kws={"s": 100}, fit_reg=True, height=6, aspect=1.6)
plt.title("Performance Over Time", fontsize=20)
plt.suptitle(f"{stock_market_cap} IPOs", fontsize=15)
plt.ylabel("YOY Change")
plt.xlabel("Days Trading")
plt.grid(which="major", axis='both', color='#e6e6e6', zorder=0)


In [ ]:
# remove stocks with None or blank as sector / industry
ax1 = sa_df.plot(kind="scatter", x="market_cap", y="first_close_to_rec_close", c=colors[0], figsize=(8,7), zorder=3)
ax.margins(0.05)

In [ ]:
sns.relplot(
    data=sa_df,
    x="days_trading", y="first_close_to_rec_close", col="market_cap_classification", col_wrap=3,
    hue="Stock Sector", palette=colors[0:3]
)